# Lab 04: Functional spatial image processing

The purpose of this lab is to practice registration steps that transform a functional image to higher-resolution anatomical space of the same subject, and then to standard space. 

## Preparation
* Import python tools for plotting 3D images
* Copy images from Lab 03 for our sub-97 example that we've been working with

In [1]:
%matplotlib inline
from nilearn import plotting
from nilearn import image

In [20]:
%%bash
# Copy files from previous labs so it is easier work with them here

# Files from 03 Lab that include our transforms for T1->MNI
cp ../03-Lab/my_T1_brain.nii.gz my_T1_brain.nii.gz
cp ../03-Lab/my_T1.nii.gz my_T1.nii.gz
cp ../03-Lab/MNI152_T1_2mm_brain.nii.gz MNI152_T1_2mm_brain.nii.gz
cp ../03-Lab/MNI152_T1_2mm.nii.gz MNI152_T1_2mm.nii.gz
cp ../03-Lab/T1_2_MNI.mat T1_2_MNI.mat # affine
cp ../03-Lab/T1_2_MNI_warp.nii.gz T1_2_MNI_warp.nii.gz # non-linear warp
cp ../03-Lab/my_T1_brain_seg_2.nii.gz my_T1_brain_wmseg.nii.gz 

# Copy functional image from osfshare
cp ../01-Lab/osfshare/sub-97_task-flanker_bold.nii.gz my_bold.nii.gz

## Registration of functional image to T1

Summary of steps 
* Extract reference volume from the 4D functional image data set 
    * fslroi tool is handy for this
   
* Estimation: 
    * Boundary Based Registration (BBR) which in theory is most accurate but takes a bit of time to run (try at home)
    * 6 degree of freedom (DOF) co-registration (faster, practice in class)
    
* Apply transform
    * applywarp

In [4]:
# There are 315 volumes in this functional data set, it doesn't make a big deal whether you round up or down
315/2

157.5

In [5]:
%%bash
fslroi


Usage: fslroi <input> <output> <xmin> <xsize> <ymin> <ysize> <zmin> <zsize>
       fslroi <input> <output> <tmin> <tsize>

       fslroi <input> <output> <xmin> <xsize> <ymin> <ysize> <zmin> <zsize> <tmin> <tsize>
Note: indexing (in both time and space) starts with 0 not 1! Inputting -1 for a size will set it to the full image extent for that dimension.


In [6]:
%%bash

# Example usage to name our middle volume as example_func
# Reads: take input of my_bold, give back example func after extracting volumes starting at vol 157 with length 1
# Remember since indexing of volumes starts at zero, this is the 158th volume
fslroi my_bold.nii.gz example_func.nii.gz 157 1

In [7]:
%%bash

# Take a look with fslview
# Decent gray matter/white matter contrast?
fslview example_func.nii.gz

In [31]:
%%bash

# Code for the 6 DOF func2T1 co-registration
flirt -in example_func \
-ref my_T1_brain \
-out func2T1_6dof \
-omat func2T1_6dof.mat \
-cost corratio \
-dof 6 \
-searchrx -90 90 -searchry -90 90 -searchrz -90 90 \
-interp trilinear


In [33]:
%%bash

# What did this produce? What is our main output of interest? 


## Problem sets

*1).* Use the epi_reg tool to generate a transform from functional to T1 space. Name your output func2T1_bbr 

In [8]:
%%bash

# Examine usage below
epi_reg


Usage: epi_reg [options] --epi=<EPI image> --t1=<wholehead T1 image> --t1brain=<brain extracted T1 image> --out=<output name>
 
Optional arguments
  --fmap=<image>         : fieldmap image (in rad/s)
  --fmapmag=<image>      : fieldmap magnitude image - wholehead extracted
  --fmapmagbrain=<image> : fieldmap magnitude image - brain extracted
  --wmseg=<image>        : white matter segmentation of T1 image
  --echospacing=<val>    : Effective EPI echo spacing (sometimes called dwell time) - in seconds
  --pedir=<dir>          : phase encoding direction, dir = x/y/z/-x/-y/-z
  --weight=<image>       : weighting image (in T1 space)
  --nofmapreg            : do not perform registration of fmap to T1 (use if fmap already registered) 
  --noclean              : do not clean up intermediate files
  -v                     : verbose output
  -h                     : display this help message
 
e.g.:  epi_reg --epi=example_func --t1=struct --t1brain=struct_brain --out=epi2struct --fmap=fmap_ra

In [29]:
%%bash

# The epi_reg command will take some time because there appears to be a bug when you supply your own _wmseg
epi_reg --epi=example_func --t1=my_T1 --t1brain=my_T1_brain --out=func2T1

Running FAST segmentation
FLIRT pre-alignment
Running BBR
0.536758 0.999835 -0.015919 -0.008741 0.000000 0.015846 0.999839 -0.008359 0.000000 0.008872 0.008219 0.999927 0.000000 -2.332991 2.286299 6.458590 1.000000 
